In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3712,  0.1710, -0.2079,  0.0097,  0.4077, -0.4502, -0.2909,  0.0235,
         -0.1094, -0.1907,  0.3264,  1.0004,  0.3694, -0.0916, -0.5161, -0.0355,
          0.4488,  0.0611,  0.4517, -0.0075],
        [ 0.0856, -0.0290, -0.0549, -0.1617,  0.2579, -0.4883,  0.1128, -0.1856,
         -0.4990, -0.4752,  0.0159,  0.5476,  0.7792, -0.5960, -0.3261, -0.0088,
          0.5872,  0.1334,  0.3111, -0.0185],
        [-0.3147,  0.4548, -0.0720,  0.1703,  0.3766,  0.0910,  0.2362, -0.1717,
         -0.2775,  0.0220,  0.0956,  0.9744,  0.2748, -0.1474,  0.0375,  0.1661,
          0.7055,  0.1923,  0.0017,  0.0207]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1710, 0.0000, 0.0097, 0.4077, 0.0000, 0.0000, 0.0235, 0.0000,
         0.0000, 0.3264, 1.0004, 0.3694, 0.0000, 0.0000, 0.0000, 0.4488, 0.0611,
         0.4517, 0.0000],
        [0.0856, 0.0000, 0.0000, 0.0000, 0.2579, 0.0000, 0.1128, 0.0000, 0.0000,
         0.0000, 0.0159, 0.5476, 0.7792, 0.0000, 0.00

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
print(logits)

tensor([[-0.2213,  0.3180, -0.2848,  0.2012, -0.1235, -0.2290,  0.2069,  0.0295,
          0.1638,  0.1238],
        [-0.2955,  0.3061, -0.3122,  0.2395, -0.1045, -0.2235,  0.2256,  0.0287,
          0.1670,  0.1476],
        [-0.3004,  0.3438, -0.2170,  0.1999, -0.2095, -0.0990,  0.2939, -0.0940,
          0.0190,  0.2309]], grad_fn=<AddmmBackward0>)


In [13]:
print(input_image)

tensor([[[0.5588, 0.4112, 0.9882,  ..., 0.5359, 0.6654, 0.6799],
         [0.6276, 0.2716, 0.5369,  ..., 0.1763, 0.1033, 0.7904],
         [0.6228, 0.4184, 0.5362,  ..., 0.2756, 0.7242, 0.3819],
         ...,
         [0.8369, 0.2509, 0.1746,  ..., 0.3107, 0.3128, 0.4862],
         [0.3824, 0.4848, 0.6790,  ..., 0.7205, 0.4166, 0.8974],
         [0.2466, 0.1296, 0.3438,  ..., 0.5143, 0.6732, 0.2673]],

        [[0.5536, 0.8216, 0.0711,  ..., 0.9247, 0.1295, 0.7379],
         [0.8370, 0.6702, 0.2223,  ..., 0.4442, 0.8981, 0.6557],
         [0.6666, 0.4784, 0.5811,  ..., 0.3033, 0.4833, 0.5876],
         ...,
         [0.5132, 0.5791, 0.3849,  ..., 0.4413, 0.7742, 0.5727],
         [0.0578, 0.1954, 0.1132,  ..., 0.9878, 0.4135, 0.4785],
         [0.3675, 0.5603, 0.7246,  ..., 0.6622, 0.0153, 0.2514]],

        [[0.3373, 0.6261, 0.3528,  ..., 0.8357, 0.9452, 0.5172],
         [0.4063, 0.3481, 0.5037,  ..., 0.9748, 0.8687, 0.5280],
         [0.4322, 0.2256, 0.9845,  ..., 0.0826, 0.7608, 0.

In [14]:
print(seq_modules)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=20, bias=True)
  (2): ReLU()
  (3): Linear(in_features=20, out_features=10, bias=True)
)


nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(softmax)

Softmax(dim=1)


In [18]:
print(pred_probab)

tensor([[0.0771, 0.1322, 0.0723, 0.1176, 0.0850, 0.0765, 0.1183, 0.0990, 0.1133,
         0.1088],
        [0.0714, 0.1302, 0.0702, 0.1218, 0.0864, 0.0767, 0.1202, 0.0987, 0.1133,
         0.1111],
        [0.0710, 0.1353, 0.0772, 0.1171, 0.0778, 0.0869, 0.1287, 0.0873, 0.0978,
         0.1208]], grad_fn=<SoftmaxBackward0>)


Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0157, -0.0032,  0.0266,  ..., -0.0146,  0.0033,  0.0167],
        [ 0.0027, -0.0153, -0.0228,  ...,  0.0286,  0.0052,  0.0295]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0034,  0.0034], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0417,  0.0431, -0.0402,  ...,  0.0040,  0.0159,  0.0172],
        [-0.0423, -0.0382, -0.0303,  ...,  0.0083,  0.0176, -0.0214]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
